<a href="https://colab.research.google.com/github/jonahyuan/AI-learning/blob/main/6_16Titanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [70]:
from urllib.request import urlretrieve
url = "https://github.com/Elwing-Chou/tibaml0606/raw/main/titanic/test.csv"
urlretrieve(url, "train.csv")
url = "https://github.com/Elwing-Chou/tibaml0606/raw/main/titanic/train.csv"
urlretrieve(url, "test.csv")

('test.csv', <http.client.HTTPMessage at 0x7f02368ce9d0>)

In [71]:
import pandas as pd
train_df = pd.read_csv("train.csv",encoding="utf-8")
predict_df = pd.read_csv("test.csv",encoding="utf-8")

In [72]:
df1 = pd.DataFrame([
     [1,2],               
      [3,4]
])
df2 = pd.DataFrame([
     [5,6,],               
      [7,8]
])
pd.concat([df1, df2], axis=0)

,0,1
0,1,2
1,3,4
0,5,6
1,7,8


In [73]:
datas = pd.concat([train_df, predict_df], axis=0, ignore_index=True)
datas = datas.drop(["PassengerId", "Survived"], axis=1)

In [74]:
df = pd.DataFrame([
    [1, 2],
    [3, 4]
])
df.sum(axis=1)

0    3
1    7
dtype: int64

In [75]:
# pandas篩選: 篩選[針對每一筆給出True/False]
df = pd.DataFrame([
    [1, 2],
    [3, 4]
])
df[[False, True]]
s = pd.Series(["A", "B", "C"])
s[[True, True, False]]

0    A
1    B
dtype: object

In [76]:
# 資料預處理
# Step1. 填補空值
# 1.1 check有多少空值
s = datas.isna().sum()
# s[s != 0]: filter操作
s[s != 0].sort_values(ascending=False)

Cabin       1014
Age          263
Embarked       2
Fare           1
dtype: int64

In [77]:
# 1.2 check每一個欄位是哪一種型態
# a.數值(1.2.3...一直增加):Age, SibSp, Parch, Ticket(多少人share), Fare 
# b.類別(固定數值):Pclass, Name(mid), Sex, Cabin(first), Embarked
# 補空值(最可能出現的值)
# 類別:補最常出現的值
# 數值:絕對不是平均值，而是中位數

In [78]:
# 處理一下 Name/Cabin/Ticket
def cabinchange(c):
  if pd.isna(c):
    return None
  else:
    return c[0]
datas["Cabin"] = datas["Cabin"].apply(cabinchange)

In [79]:
count = {"a":5, "b":3, "c":2}
s = pd.Series(["a", "b", "c"])
def change(c):
    return count[c]
s.apply(change)

0    5
1    3
2    2
dtype: int64

In [80]:
counts = datas["Ticket"].value_counts()
def ticketchange(t):
  if pd.isna(t):
    return None
  else:
    return counts[t]
datas["Ticket"] = datas["Ticket"].apply(ticketchange)

In [81]:
datas["Name"]

0                                   Kelly, Mr. James
1                   Wilkes, Mrs. James (Ellen Needs)
2                          Myles, Mr. Thomas Francis
3                                   Wirz, Mr. Albert
4       Hirvonen, Mrs. Alexander (Helga E Lindqvist)
                            ...                     
1304                           Montvila, Rev. Juozas
1305                    Graham, Miss. Margaret Edith
1306        Johnston, Miss. Catherine Helen "Carrie"
1307                           Behr, Mr. Karl Howell
1308                             Dooley, Mr. Patrick
Name: Name, Length: 1309, dtype: object

In [82]:
#n = "  Behr, Mr. Karl Howell"
def namechange(n):
  return n.split(",")[-1].split(".")[0].strip()
datas["Name"] = datas["Name"].apply(namechange)

In [83]:
# 耍帥用
import re
n = "  Behr, Mr. Karl Howell"
# .+任意一個字一次以上 ,:逗號 (.+):任意一個字一次以上，並且group起來 \.:match點 +:任意一個字以上
pat = r".+,(.+)\..+"
re.match(pat, n).group(1).strip()

'Mr'

In [84]:
most = datas["Embarked"].value_counts().idxmax()
datas["Embarked"] = datas["Embarked"].fillna(most)

In [85]:
df = pd.DataFrame([
    [1, 2, "a"],
    [None, None, None],
], columns=["D", "E", "F"])
fill = {"D":2.5, "E":3.1, "F":"bbb"}
df.fillna(fill)

,D,E,F
0,1.0,2.0,a
1,2.5,3.1,bbb


In [86]:
med = datas.median().drop(["Pclass"])
datas = datas.fillna(med)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  """Entry point for launching an IPython kernel.


In [87]:
count = datas["Name"].value_counts()
reserved = count[count > 50].index
def namefilter(mid):
  if mid in reserved:
    return mid
  else:
    return None
datas["Name"] = datas["Name"].apply(namefilter)

In [88]:
pd.set_option("display.max_columns", 30)
datas = pd.get_dummies(datas)
datas = pd.get_dummies(datas, columns=["Pclass"])
datas

,Age,SibSp,Parch,Ticket,Fare,Name_Master,Name_Miss,Name_Mr,Name_Mrs,Sex_female,Sex_male,Cabin_A,Cabin_B,Cabin_C,Cabin_D,Cabin_E,Cabin_F,Cabin_G,Cabin_T,Embarked_C,Embarked_Q,Embarked_S,Pclass_1,Pclass_2,Pclass_3
0,34.5,0,0,1,7.8292,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1
1,47.0,1,0,1,7.0000,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1
2,62.0,0,0,1,9.6875,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0
3,27.0,0,0,1,8.6625,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1
4,22.0,1,1,2,12.2875,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1304,27.0,0,0,1,13.0000,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0
1305,19.0,0,0,1,30.0000,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0
1306,28.0,1,2,4,23.4500,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1
1307,26.0,0,0,1,30.0000,0,0,1,0,0,1,0,0,1,0,0,0,0,0,1,0,0,1,0,0


In [89]:
datas["Family"] = datas["SibSp"] = datas["Parch"]
datas

,Age,SibSp,Parch,Ticket,Fare,Name_Master,Name_Miss,Name_Mr,Name_Mrs,Sex_female,Sex_male,Cabin_A,Cabin_B,Cabin_C,Cabin_D,Cabin_E,Cabin_F,Cabin_G,Cabin_T,Embarked_C,Embarked_Q,Embarked_S,Pclass_1,Pclass_2,Pclass_3,Family
0,34.5,0,0,1,7.8292,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0
1,47.0,0,0,1,7.0000,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0
2,62.0,0,0,1,9.6875,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0
3,27.0,0,0,1,8.6625,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0
4,22.0,1,1,2,12.2875,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1304,27.0,0,0,1,13.0000,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0
1305,19.0,0,0,1,30.0000,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,0
1306,28.0,2,2,4,23.4500,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,2
1307,26.0,0,0,1,30.0000,0,0,1,0,0,1,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0
